## Парсер сайта по подбору психологов Alter
На какие параметры обращаем внимание:

* Имя, фамилия
* Цена за сессию (руб)
*  Город
*  Ведет ли онлайн (0,1)
*  Подход, в котором работает (может быть несколько)
* Специализация (основные запросы, в форме списка)
* Опыт работы (в годах)
* Возраст
* Семейное положение (категориальное)
* Пол  (0,1)
* Наличие детей
* Работает ли с верующими  (0,1)
* Является ли фем-френдли  (0,1)
* Является ли квир-френдли  (0,1) (обозначено как 💗-френдли)
* Работает ли с онко-пациентами (0,1)
* Квалифиация: образование (текст)
* Наличие видео-приветствия  (0,1)
* Наличие аудио-приветствия (0,1)
* Раздел "О себе" (текст)

In [ ]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
from tqdm import tqdm

# Парсим ссылки на психологов

In [ ]:
links = []
def parse_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    cards = soup.find_all('div', class_='psychologist-list__psychologist psychologist-card')  #ищем все кнопочки "подробнее"
    for card in cards:
        link = card.find('a')['href']  # из них забираем ссылочку
        full_link = f"https://alter.ru{link}"  # кидаем в массив полную ссылку на страницу психолога
        links.append(full_link)

    return links

parse_page('https://alter.ru/psychologists')

[]

Выяснилось что страницы появляются динамически и реквест не справился. Плачем, смотрим мем, подключаем селениум

<a href="https://ibb.co/dwfjphb0"><img src="https://i.ibb.co/W4HGzTDW/IMG-5596.jpg" alt="IMG-5596" border="0"></a>


In [ ]:
!pip install selenium
!apt-get update  # Обновляем пакеты
!apt install -y chromium-chromedriver  # Устанавливаем ChromeDriver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 24.9 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,321 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,708 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-driv

In [ ]:
# импорт всего на свете
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
from tqdm import tqdm

chrome_options = Options()

chrome_options.page_load_strategy = 'eager'
chrome_options.add_argument('--headless')
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument('--disable-gpu')


driver = webdriver.Chrome(options=chrome_options)


После такого количества грустного текста срочно поднимаем настроение картинкой

<a href="https://ibb.co/zVCwKLKN"><img src="https://i.ibb.co/3mVHt8tk/IMG-5602.jpg" alt="IMG-5602" border="0"></a>

In [ ]:
def parse_page(url, links):
    try:
        driver.get(url) #заходим на страничку и ждем загрузки или 15 секунд
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div.psychologist-list__psychologist.psychologist-card'))
        )

        # ищем все карточки на странице
        cards = driver.find_elements(By.CSS_SELECTOR, 'div.psychologist-list__psychologist.psychologist-card')
        links = []
        for card in cards:
            try:
                link = card.find_element(By.CSS_SELECTOR, 'a.psychologist').get_attribute('href')
                links.append(link)
            except Exception as e:
                print(f"Ошибка при поиске ссылки в карточке: {e}")

        return links

    except TimeoutException as e: #если не нашли такую страничку то дальше просто идем и регистрируем ошибку
        print(f"Ошибка при парсинге страницы {url}: {e}")
        return None  # возвращаем None, чтобы обозначить ошибку
    return links

Парсим линки на всех психологов на платформе

In [ ]:
driver = webdriver.Chrome(options=chrome_options)
errors = []
links = set()
while len(links) < 876: # пояснение почему тут нужен while будет ниже в парсинге доп.массивов
    driver.get("https://alter.ru/psychologists/1")
    for i in tqdm(range(1, 147), desc="Парсинг ссылок", unit="страница"): #количество страниц на сайте тут и далее определяли ручками на момент 28.02.25
        # Парсим текущую страницу
        new_links = [a.split('?')[0] for a in parse_page(driver.current_url, links)]
        links.update(new_links)

        # Переход на следующую страницу
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "li.page-item a.page-link[href='#next']"))
            )
            if "disabled" in next_button.get_attribute("class"):
                break  # Если кнопка "Далее" неактивна, завершаем
            next_button.click()
        except:
            break  # Если кнопка "Далее" не найдена, завершаем
            errors += driver.current_url
    print(f"Найдено {len(links)} ссылок")


Парсинг ссылок:  99%|█████████▉| 145/146 [12:19<00:05,  5.10s/страница]


Найдено 716 ссылок


Парсинг ссылок:  99%|█████████▉| 145/146 [11:50<00:04,  4.90s/страница]


Найдено 833 ссылок


Парсинг ссылок:  99%|█████████▉| 145/146 [12:05<00:05,  5.00s/страница]


...

Найдено 872 ссылок


Парсинг ссылок: 100%|██████████| 146/146 [11:43<00:00,  4.82s/страница]


Найдено 875 ссылок


Парсинг ссылок: 100%|██████████| 146/146 [12:28<00:00,  5.13s/страница]

Найдено 877 ссылок


Ну мы че-то спарсили, теперь по каждой ссылочке обработаем психологов

Экспортнем все ссылки

In [ ]:
links = list(links)
with open("links.txt", "w", encoding="utf-8") as file:
    for link in links:
        file.write(f"{link}\n")

# Выделяем штуки из кода страницы

In [ ]:
driver = webdriver.Chrome(options=chrome_options)

url = 'https://alter.ru/psychologists/budva/ekaterina-fimina' #будем тестить все на ней, потому что у одной из нас маму зовут Екатерина Зимина
driver.get(url)
WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'h1.style_title__rerZD')))
name = driver.find_element(By.CSS_SELECTOR, 'h1.style_title__rerZD').text
name

'Екатерина Фимина'

Стоимость

In [ ]:
driver.get(url)
WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'span.style_price__fTH3C span.is-nowrap')))
price = int(driver.find_element(By.CSS_SELECTOR, 'span.style_price__fTH3C span.is-nowrap').text.replace(' ₽', ''))
price

3500

Возраст и опыт

In [ ]:
WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'span.style_experience__CluEG')))
age = int(driver.find_element(By.CSS_SELECTOR, 'span.style_age__hYhw7').text.split()[0])
experience = int(driver.find_element(By.CSS_SELECTOR, 'span.style_experience__CluEG').text.split()[0])
age, experience

(59, 7)

Принимает онлайн и лично

In [ ]:
WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'span.style_location__DucmB')))
online = (1 if 'онлайн' in driver.find_element(By.CSS_SELECTOR, 'span.style_location__DucmB').text else 0)
in_person = (1 if 'лично' in driver.find_element(By.CSS_SELECTOR, 'span.style_location__DucmB').text else 0)
online, in_person

(1, 0)

Город

In [ ]:
WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'div.style_city__ANaA_ span.style_blmText__6echM')))
city = driver.find_element(By.CSS_SELECTOR, 'div.style_city__ANaA_ span.style_blmText__6echM').text
city

'Будва'

Подход

In [ ]:
WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'div.style_list__ro209')))
approaches = driver.find_elements(By.CSS_SELECTOR, 'span.style_name__9qe2P')
approach_list = [approach.text for approach in approaches]

Психолог о себе

In [ ]:
url = 'https://alter.ru/psychologists/kostroma/nadezhda-golubeva'
driver.get(url)
try:
    # Ждем загрузки контейнера с абзацами
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.style_paragraphs__AXJr2'))
    )

    # Прокручиваем страницу до кнопки "Показать больше"
    show_more_button = driver.find_element(By.CSS_SELECTOR, 'span[data-testid="psy-profile-about-test-more"]')
    driver.execute_script("arguments[0].scrollIntoView(true);", show_more_button)
    time.sleep(1)  # Ждем завершения прокрутки

    # Кликаем по кнопке с помощью JavaScript
    driver.execute_script("arguments[0].click();", show_more_button)

    # Ждем обновления списка
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.style_paragraphs__AXJr2 span.style_blmText__6echM'))
    )

    # Ищем все элементы с текстом
    about = driver.find_elements(By.CSS_SELECTOR, 'div.style_paragraphs__AXJr2 span.style_blmText__6echM')
    about =' '.join([abouts.text for abouts in about])
except Exception as e:
    print(f"Ошибка при парсинге абзацев: {e}")

about

'Опираясь на свой клиентский опыт и опыт моих клиентов, могу сказать, что психотерапия помогает повысить качество жизни. В чем это проявляется? Вы сможете лучше понимать себя и свои желания и потребности, находить способы и ресурсы достигать цели или удовлетворять эти потребности, или прожить невозможность их достижения/удовлетворения.Конечно, психотерапия не сделает это чудесным образом, клиент сам прилагает к этому усилия. Для меня психотерапия похожа на образ заблудившегося в лесу человека, и моя роль – это не проводник, который знает куда идти и выведет его из леса. Психолог – это скорее спутник/партнер, который будет сопровождать клиента в процессе нахождения своего пути. Я убеждена, что только сам клиент знает, куда ему надо. Моя задача как психолога, помочь найти этот путь, обратить внимание на что-то, что ранее клиент не замечал, остановиться и оглядеться там, где ранее не останавливался. Я предпочитаю выстраивать с клиентом долговременную работу. В процессе такой работы получа

Фем, квир, онко, верующие (ААААААААААААААААААААА)

Тут зафиксированы страдания: если открыть страничку с включенным фильтром то в кэше браузера останется фильтр (френдли, фем-френдли и т.д.), но для драйвера этого кэша нет, и мы час разбирались почему он игнорирует часть кода элемента:


In [ ]:
driver = webdriver.Chrome(options=chrome_options)
url = 'https://alter.ru/psychologists/msk/andrei-reshetarov'
driver.get(url)

def check_text_in_labels(text):
    try:
        # Используем XPath для поиска div с классом labels__label, содержащего нужный текст
        WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-testid="psy-profile-labels"]')))
        element = driver.find_element(By.XPATH, f"//div[contains(@class, 'labels__label') and contains(text(), '{text}')]")
        return True
    except:
        return False

# Проверяем наличие текста "Френдли" и "Фем-френдли"
result_frendly = check_text_in_labels("Френдли")
result_fem_frendly = check_text_in_labels("Фем-френдли")

result_frendly, result_fem_frendly


(False, False)

Специализация

In [ ]:
WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-testid="psy-profile-competences-help-title"]')))

try:
    show_more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'span[data-testid="psy-profile-competences-help-more"]'))
        )

        # Прокрутка страницы до кнопки
    driver.execute_script("arguments[0].scrollIntoView(true);", show_more_button)
    time.sleep(1)  # Ждем завершения прокрутки

        # Кликаем по кнопке с помощью JavaScript
    driver.execute_script("arguments[0].click();", show_more_button)

        # Ждем обновления списка
    WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div[data-testid="psy-profile-competences-help-title"] + div.style_container__v04cR span.style_blmText__6echM'))
        )
except Exception:
    b = 0
WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-testid="psy-profile-competences-help-title"]'))
    )
phrases = driver.find_elements(By.CSS_SELECTOR, 'div[data-testid="psy-profile-competences-help-title"] + div.style_container__v04cR span.style_blmText__6echM')

    # Извлечение текста фраз
phrase_list = [phrase.text for phrase in phrases][:-1]
phrase_list

['Самооценка и уверенность в себе',
 'Сложные перемены и жизненные ситуации',
 'Одиночество и отсутствие отношений',
 'Отношения с родителями',
 'Расставание',
 'Прокрастинация и личная эффективность',
 'Трудоголизм и work-life balance',
 'Работа с эмоциями',
 'Трудный прошлый опыт',
 'Отношения с партнером',
 'Отношения с друзьями, коллегами и другими людьми',
 'Эмоциональное и профессиональное выгорание',
 'Смысл жизни, жизненные цели, самоопределение']

Образование:

In [ ]:
url = 'https://alter.ru/psychologists/kostroma/nadezhda-golubeva'
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)
WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.mb-4 span.style_title__F7cV2'))
    )

    # Попытка найти и нажать кнопку "Свернуть" (если она есть)
try:
    show_more_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'span.style_showMore__uyGKl'))
        )
    driver.execute_script("arguments[0].scrollIntoView(true);", show_more_button)
    time.sleep(1)  # Ждем завершения прокрутки
    driver.execute_script("arguments[0].click();", show_more_button)

        # Ждем обновления списка
    WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.mb-4 ul.list-disc li span.font-futura-400'))
        )
except Exception:
    education_elements = None

    # Поиск всех элементов с текстом об образовании
education_elements = driver.find_elements(By.CSS_SELECTOR, 'div.mb-4 ul.list-disc li span.font-futura-400')

    # Извлечение текста об образовании
education_list = [element.text.strip() for element in education_elements]
print(f"Образование: {education_list}")

Образование: ['Костромской государственный университет им. Н.А. Некрасова. «Психолог. преподаватель психологии».', 'Ярославский государственный университет им. П.Г. Демидова. «Психология» — Магистр ( диплом с отличием).']


С кем работает:

In [ ]:
WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-testid="psy-profile-competences-work-title"]'))
    )

    # Поиск всех элементов с фразами после "С кем работает"
work_with_elements = driver.find_elements(By.CSS_SELECTOR, 'div.styles_emoji__kMn91 span.style_blmText__6echM.style_accent__CgD_s')

    # Извлечение текста фраз
work_with_list = [element.text for element in work_with_elements]
print(f"С кем работает: {work_with_list}")

С кем работает: ['Взрослые']


Парсим всех фем-френдли (мы вернулись к этому и стали мэтчить несколько массивов вместо попыток напрямую вытащить из сайта). Массивы не супер биг дата, так что можем себе позволить.

Сразу после этого сто лет пытались сделать так, чтобы сессия не сбрасывалась (не перемешивались психологи после перехода на следующую страницу сайта), но даже куки не помогли и сайт на каждой странице выкидывал 6 рандомных психологов, из-за чего у нас были повторки в массивах(((

Поэтому парсим все, пока не найдем уникальное и нужное количество психологов

In [ ]:
driver = webdriver.Chrome(options=chrome_options)
femme = set()
while len(femme) < 62:
    driver.get("https://alter.ru/psychologists/1?working_with.fem=true")
    for i in tqdm(range(1, 12), desc="Парсинг ссылок фем френдли", unit="страница"):
        # Парсим текущую страницу
        new_links = [a.split('?')[0] for a in parse_page(driver.current_url, femme)]
        femme.update(new_links)

        # Переход на следующую страницу
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "li.page-item a.page-link[href='#next']"))
            )
            if "disabled" in next_button.get_attribute("class"):
                break  # Если кнопка "Далее" неактивна, завершаем
            next_button.click()
        except:
            break  # Если кнопка "Далее" не найдена, завершаем


Парсинг ссылок фем френдли:  91%|█████████ | 10/11 [00:53<00:05,  5.34s/страница]


Запишем все это в файлик

In [ ]:
femme = list(femme)
with open("femme.txt", "w", encoding="utf-8") as file:
    for index, link in enumerate(femme, start=1):
        file.write(f"{index}. {link}\n")

Парсим всех квир френдли

In [ ]:
driver = webdriver.Chrome(options=chrome_options)
friendly = set()
while len(friendly) < 207:
    driver.get("https://alter.ru/psychologists/1?working_with.friendly=true")
    for i in tqdm(range(1, 36), desc="Парсинг ссылок квир френдли", unit="страница"):
        # Парсим текущую страницу
        new_links = [a.split('?')[0] for a in parse_page(driver.current_url, friendly)]
        friendly.update(new_links)

        # Переход на следующую страницу
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "li.page-item a.page-link[href='#next']"))
            )
            if "disabled" in next_button.get_attribute("class"):
                break  # Если кнопка "Далее" неактивна, завершаем
            next_button.click()
        except:
            break  # Если кнопка "Далее" не найдена, завершаем

print(f"Найдено {len(friendly)} ссылок")

Парсинг ссылок квир френдли:  97%|█████████▋| 34/35 [02:49<00:04,  5.00s/страница]

Найдено 207 ссылок


In [ ]:
friendly = list(friendly)
with open("friendly.txt", "w", encoding="utf-8") as file:
    for index, link in enumerate(friendly, start=1):
        file.write(f"{index}. {link}\n")

Парсим всех, кто работает с онкобольными

In [ ]:
driver = webdriver.Chrome(options=chrome_options)
cancer = set()
while len(cancer) < 97:
    driver.get("https://alter.ru/psychologists/1?working_with.cancer=true")
    for i in tqdm(range(1, 18), desc="Парсинг ссылок работающих с онкопациентами", unit="страница"):
        # Парсим текущую страницу
        new_links = [a.split('?')[0] for a in parse_page(driver.current_url, cancer)]
        cancer.update(new_links)

        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "li.page-item a.page-link[href='#next']"))
            )
            if "disabled" in next_button.get_attribute("class"):
                break  # Если кнопка "Далее" неактивна, завершаем
            next_button.click()
        except:
            break  # Если кнопка "Далее" не найдена, завершаем
    print(f"Найдено {len(cancer)} ссылок")


Парсинг ссылок работающих с онкопациентами:  94%|█████████▍| 16/17 [01:39<00:06,  6.25s/страница]


Найдено 93 ссылок


Парсинг ссылок работающих с онкопациентами:  94%|█████████▍| 16/17 [01:35<00:05,  6.00s/страница]


Найдено 96 ссылок


Парсинг ссылок работающих с онкопациентами:  94%|█████████▍| 16/17 [01:31<00:05,  5.74s/страница]

Найдено 97 ссылок


In [ ]:
cancer = list(cancer)
with open("cancer.txt", "w", encoding="utf-8") as file:
    for link in cancer:
        file.write(f"{link}\n")

Парсим всех, кто работает с верующими

In [ ]:
driver = webdriver.Chrome(options=chrome_options)
believer = set()
while len(believer) < 18:
    driver.get("https://alter.ru/psychologists/1?working_with.believer=true")
    for i in tqdm(range(1, 4), desc="Парсинг ссылок работающих с верующими", unit="страница"):
        # Парсим текущую страницу
        new_links = [a.split('?')[0] for a in parse_page(driver.current_url, believer)]
        believer.update(new_links)

        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "li.page-item a.page-link[href='#next']"))
            )
            if "disabled" in next_button.get_attribute("class"):
                break  # Если кнопка "Далее" неактивна, завершаем
            next_button.click()
        except:
            break  # Если кнопка "Далее" не найдена, завершаем
    print(f"Найдено {len(believer)} ссылок")

Парсинг ссылок работающих с верующими:  67%|██████▋   | 2/3 [00:18<00:09,  9.08s/страница]

Найдено 18 ссылок


In [ ]:
believer = list(believer)
with open("believer.txt", "w", encoding="utf-8") as file:
    for link in believer:
        file.write(f"{link}\n")

Парсим пол

In [ ]:
male = set()
while len(male) < 128:
    driver.get("https://alter.ru/psychologists/1?sex=M")
    for i in tqdm(range(1, 23), desc="Парсинг мужчин", unit="страница"):
    # Парсим текущую страницу
        new_links = [a.split('?')[0] for a in parse_page(driver.current_url, male)]
        male.update(new_links)

        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.page-link[href="#next"]')))
            next_button.click()
            WebDriverWait(driver, 10).until(EC.staleness_of(next_button))
        except Exception as e:
            break

print(f"Найдено {len(male)} ссылок")

Парсинг мужчин:  95%|█████████▌| 21/22 [01:40<00:04,  4.81s/страница]

Найдено 128 ссылок


In [ ]:
male = list(male)
with open("male.txt", "w", encoding="utf-8") as file:
    for link in male:
        file.write(f"{link}\n")

Парсим детей

<a href="https://ibb.co/zHjDzz59"><img src="https://i.ibb.co/qFXVvvnq/IMG-5594.jpg" alt="IMG-5594" border="0"></a>


In [ ]:
kids = []
no_kids = []

driver.get("https://alter.ru/psychologists/1?has_children=no")
for i in tqdm(range(1, 53), desc="Парсинг ссылок психологов без детей", unit="страница"): #страниц c психологами без детей 52
    no_kids += parse_page(driver.current_url, no_kids)
    driver.get(f"https://alter.ru/psychologists/{i + 1}?has_children=no")

print(f"Найдено {len(no_kids)} ссылок")


driver.get("https://alter.ru/psychologists/1?has_children=yes")
for i in tqdm(range(1, 74), desc="Парсинг ссылок психологов с детьми", unit="страница"): #страниц с психологами-родителями  73
    kids += parse_page(driver.current_url, kids)
    driver.get(f"https://alter.ru/psychologists/{i + 1}?has_children=yes")

print(f"Найдено {len(kids)} ссылок")

Парсинг ссылок психологов без детей: 100%|██████████| 52/52 [07:54<00:00,  9.13s/страница]


Найдено 310 ссылок


Парсинг ссылок психологов с детьми: 100%|██████████| 73/73 [11:03<00:00,  9.09s/страница]

Найдено 438 ссылок


In [ ]:
no_kids = set()

while len(no_kids) < 308:
    driver.get("https://alter.ru/psychologists/1?has_children=no")
    for i in tqdm(range(1, 53), desc="Парсинг ссылок психологов без детей", unit="страница"):
    # Парсим текущую страницу
        new_links = [a.split('?')[0] for a in parse_page(driver.current_url, no_kids)]
        no_kids.update(new_links)

        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.page-link[href="#next"]')))
            next_button.click()
            WebDriverWait(driver, 10).until(EC.staleness_of(next_button))
        except Exception as e:
            break

print(f"Найдено {len(no_kids)} ссылок")

Парсинг ссылок психологов без детей:  98%|█████████▊| 51/52 [03:54<00:04,  4.60s/страница]

Найдено 308 ссылок


In [ ]:
no_kids = list(no_kids)
with open("no_kids.txt", "w", encoding="utf-8") as file:
    for link in no_kids:
        file.write(f"{link}\n")

In [ ]:
kids = set()

while len(kids) < 437:
    driver.get("https://alter.ru/psychologists/1?has_children=yes")
    for i in tqdm(range(1, 74), desc="Парсинг ссылок психологов c детьми", unit="страница"):
    # Парсим текущую страницу
        new_links = [a.split('?')[0] for a in parse_page(driver.current_url, kids)]
        kids.update(new_links)

        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.page-link[href="#next"]')))
            next_button.click()
            WebDriverWait(driver, 10).until(EC.staleness_of(next_button))
        except Exception as e:
            break

print(f"Найдено {len(kids)} ссылок")

Парсинг ссылок психологов c детьми:  99%|█████████▊| 72/73 [05:44<00:04,  4.79s/страница]

Найдено 437 ссылок


In [ ]:
kids = list(kids)
with open("kids.txt", "w", encoding="utf-8") as file:
    for link in kids:
        file.write(f"{link}\n")

Парсим наличие видеопривествия

In [ ]:
driver.get("https://alter.ru/psychologists/spb/valeriia-soloveva") #здесь в функцию покидали разных психологов, чтобы убедиться что все работает
try:                                                                #и оно реально работает
    video_element = WebDriverWait(driver, 1).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.style_video__oSOnM")))
    print(1)
except TimeoutException:
    print(0)

0


In [ ]:
driver.get("https://alter.ru/psychologists/spb/valeriia-soloveva") #аналогично - это тестовая ячейка
try:
    video_element = WebDriverWait(driver, 1).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.style_audio__8T4C1")))
    print(1)
except TimeoutException:
    print(0)

0


Парсим семейное положение

In [ ]:
married = set()

while len(married) < 516:
    driver.get("https://alter.ru/psychologists/1?marital_status=married")
    for i in tqdm(range(1, 87), desc="Парсинг ссылок психологов со статусом Женат / Замужем", unit="страница"):
    # Парсим текущую страницу
        new_links = [a.split('?')[0] for a in parse_page(driver.current_url, married)]
        married.update(new_links)

        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.page-link[href="#next"]')))
            next_button.click()
            WebDriverWait(driver, 10).until(EC.staleness_of(next_button))
        except Exception as e:
            break

print(f"Найдено {len(married)} ссылок")

Парсинг ссылок психологов со статусом Женат / Замужем:  99%|█████████▉| 85/86 [06:46<00:04,  4.78s/страница]
Парсинг ссылок психологов со статусом Женат / Замужем:  99%|█████████▉| 85/86 [06:49<00:04,  4.82s/страница]
Парсинг ссылок психологов со статусом Женат / Замужем:  99%|█████████▉| 85/86 [06:44<00:04,  4.76s/страница]
Парсинг ссылок психологов со статусом Женат / Замужем:  99%|█████████▉| 85/86 [06:35<00:04,  4.65s/страница]

Найдено 516 ссылок


In [ ]:
single = set()

while len(single) < 224:
    driver.get("https://alter.ru/psychologists/1?marital_status=single")
    for i in tqdm(range(1, 39), desc="Парсинг ссылок психологов со статусом не Женат /не Замужем", unit="страница"):
    # Парсим текущую страницу
        new_links = [a.split('?')[0] for a in parse_page(driver.current_url, single)]
        single.update(new_links)

        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.page-link[href="#next"]')))
            next_button.click()
            WebDriverWait(driver, 10).until(EC.staleness_of(next_button))
        except Exception as e:
            break

print(f"Найдено {len(single)} ссылок")

Парсинг ссылок психологов со статусом не Женат /не Замужем:  97%|█████████▋| 37/38 [02:55<00:04,  4.75s/страница]
Парсинг ссылок психологов со статусом не Женат /не Замужем:  97%|█████████▋| 37/38 [02:51<00:04,  4.63s/страница]
Парсинг ссылок психологов со статусом не Женат /не Замужем:  97%|█████████▋| 37/38 [02:53<00:04,  4.69s/страница]
Парсинг ссылок психологов со статусом не Женат /не Замужем:  97%|█████████▋| 37/38 [02:54<00:04,  4.71s/страница]

Найдено 224 ссылок


In [ ]:
single = list(single)
with open("single.txt", "w", encoding="utf-8") as file:
    for link in single:
        file.write(f"{link}\n")

In [ ]:
married = list(married)
with open("married.txt", "w", encoding="utf-8") as file:
    for link in married:
        file.write(f"{link}\n")

# Парсим инфу о психологах

Наконец-то!!!! Итоговый парсинг

In [ ]:
def read_links(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        links = [line.strip() for line in file.readlines()]
    return links

In [ ]:
error = []

In [ ]:
def parse_psychologist_page(url):
    driver.get(url)
    WebDriverWait(driver, 10).until(
        lambda d: d.execute_script("return document.readyState") == "complete")
    # Инициализация данных
    data = {
        "Имя фамилия": "",
        "Цена за сессию (руб)": "",
        "Город": "",
        "Онлайн": 0,
        "Лично": 0,
        "Подходы": [],
        "Специализация": [],
        "С кем работает": [],
        "Опыт работы (в годах)": "",
        "Возраст (в годах)": "",
        "Семейное положение": None,
        "Пол": 0,
        "Наличие детей": None,
        "Работает ли с верующими": 0,
        "Фем-френдли": 0,
        "Квир-френдли": 0,
        "Работает ли с онко-пациентами": 0,
        "Образование": "",
        "Наличие видеоприветствия": 0,
        "Наличие аудиоприветствия": 0,
        "О себе":  ''
    }
    try:
        # Имя фамилия (готово и работает)
        name = driver.find_element(By.CSS_SELECTOR, 'h1.style_title__rerZD').text
        data["Имя фамилия"] = name

        # Цена за сессию (готово и работает)
        try:
            price = int(driver.find_element(By.CSS_SELECTOR, 'span.style_price__fTH3C span.is-nowrap').text.replace(' ₽', '').replace(',','').replace('.',''))
            data["Цена за сессию (руб)"] = price
        except Exception as e:
            data["Цена за сессию (руб)"] = None

        # Город (готово)
        city = driver.find_element(By.CSS_SELECTOR, 'div.style_city__ANaA_ span.style_blmText__6echM').text
        data["Город"] = city.split(',')[0]

        # Онлайн (готово)
        online =  (1 if 'онлайн' in driver.find_element(By.CSS_SELECTOR, 'span.style_location__DucmB').text else 0)
        data["Онлайн"] = online

        # Лично (готово)
        in_person =  (1 if 'лично' in driver.find_element(By.CSS_SELECTOR, 'span.style_location__DucmB').text else 0)
        data["Лично"] = in_person

        # Подход, в котором работает
        approaches = driver.find_elements(By.CSS_SELECTOR, 'span.style_name__9qe2P')
        approach_list = [approach.text for approach in approaches]
        data['Подходы'] = approach_list

        # Специализация
        try:
            show_more_button = WebDriverWait(driver, 2).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, 'span[data-testid="psy-profile-competences-help-more"]')))
            driver.execute_script("arguments[0].scrollIntoView(true);", show_more_button)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", show_more_button)
            WebDriverWait(driver, 2).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div[data-testid="psy-profile-competences-help-title"] \
                    + div.style_container__v04cR span.style_blmText__6echM')))
        except Exception:
            phrases = None

        phrases = driver.find_elements(By.CSS_SELECTOR, 'div[data-testid="psy-profile-competences-help-title"] + div.style_container__v04cR span.style_blmText__6echM')
        phrase_list = [phrase.text for phrase in phrases][:-1]
        data["Специализация"] = phrase_list

        # Опыт работы (готово)
        try:
            experience = int(driver.find_element(By.CSS_SELECTOR, 'span.style_experience__CluEG').text.split()[0])
            data["Опыт работы (в годах)"] = experience
        except Exception as e:
            data["Опыт работы (в годах)"] = 1

        # Возраст (готово)
        try:
            age = int(driver.find_element(By.CSS_SELECTOR, 'span.style_age__hYhw7').text.split()[0])
            data["Возраст (в годах)"] = age
        except Exception as e:
            data['Возраст (в годах)'] = None

        #С кем работает (готово)
        work_with_elements = driver.find_elements(By.CSS_SELECTOR, 'div.styles_emoji__kMn91 span.style_blmText__6echM.style_accent__CgD_s')
        work_with_list = [element.text for element in work_with_elements]
        data['С кем работает'] = work_with_list

        # Семейное положение (готово)
        matches_mar = read_links('married.txt')
        matches_sin = read_links('single.txt')
        data["Семейное положение"] = 1 if url in matches_mar else 0 if url in matches_sin else None

        # Пол (готово)
        male = read_links('male.txt')
        data["Пол"] = 1 if url in male else 0

        # Наличие детей (готово)
        matches_k = read_links('kids.txt')
        matches_nk = read_links('no_kids.txt')
        data["Наличие детей"] = 1 if url in matches_k else 0 if  url in matches_nk else None

        # Работает ли с верующими (готово)
        believer = read_links('believer.txt')
        data["Работает ли с верующими"] = 1 if url in believer else 0

        # Фем-френдли (готово)
        matches = [a.split('. ')[1] for a in read_links('femme.txt')] #тут чуть-чуть по тупому спарсили ссылки с нумерацией и было лень переделывать((
        data["Фем-френдли"] = 1 if url in matches else 0

        # Квир-френдли (готово)
        matches = [a.split('. ')[1] for a in read_links('friendly.txt')]
        data["Квир-френдли"] =  1 if url in matches else 0

        # Работает ли с онко-пациентами (готово)
        matches = read_links('cancer.txt')
        data["Работает ли с онко-пациентами"] = 1 if url in matches else 0

        # Образование (готово)
        education_elements = []
        WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.mb-4 span.style_title__F7cV2'))
            )

            # Попытка найти и нажать кнопку "Свернуть" (если она есть)
        try:
            show_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'span.style_showMore__uyGKl'))
                )
            driver.execute_script("arguments[0].scrollIntoView(true);", show_more_button)
            time.sleep(1)  # Ждем завершения прокрутки
            driver.execute_script("arguments[0].click();", show_more_button)

                # Ждем обновления списка
            WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.mb-4 ul.list-disc li span.font-futura-400'))
                )
        except Exception:
            education_elements = None

            # Поиск всех элементов с текстом об образовании
        education_elements = driver.find_elements(By.CSS_SELECTOR, 'div.mb-4 ul.list-disc li span.font-futura-400')

            # Извлечение текста об образовании
        education_list = [element.text.strip() for element in education_elements]
        data['Образование'] = education_list

        # Раздел "О себе"
        try:
            # есть показать больше?
            show_more_button = driver.find_element(By.CSS_SELECTOR, 'span[data-testid="psy-profile-about-test-more"]')
            # если да то клик
            driver.execute_script("arguments[0].scrollIntoView(true);", show_more_button)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", show_more_button)  # Клик по кнопке

            # обновляем
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.style_paragraphs__AXJr2 span.style_blmText__6echM'))
            )
        except Exception as e:
            #если кнопки нет то как бы и окей
            about = None
        about_elements = driver.find_elements(By.CSS_SELECTOR, 'div.style_paragraphs__AXJr2 span.style_blmText__6echM')
        about = ' '.join([element.text for element in about_elements])
        data['О себе'] = about

        # Наличие видеоприветствия (готово)
        try:
            video_element = WebDriverWait(driver, 1).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.style_video__oSOnM")))
            data['Наличие видеоприветствия'] = 1
        except TimeoutException:
            data['Наличие видеоприветствия'] = 0

        # Наличие аудиоприветствия (готово)
        try:
            video_element = WebDriverWait(driver, 1).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.style_audio__8T4C1")))
            data['Наличие аудиоприветствия'] = 1
        except TimeoutException:
            data['Наличие аудиоприветствия'] = 0
    except Exception as e:
        print(f"Ошибка при парсинге страницы {url}:{e}")
    return data

In [ ]:
parse_psychologist_page('https://alter.ru/psychologists/msk/miroslav-iasin') #тестим на одном человечке

{'Имя фамилия': 'Мирослав Ясин',
 'Цена за сессию (руб)': 3900,
 'Город': 'Москва',
 'Онлайн': 1,
 'Лично': 0,
 'Подходы': ['КПТ', 'Позитивная', 'КЦП'],
 'Специализация': ['Самооценка и уверенность в себе',
  'Эмоциональное и профессиональное выгорание',
  'Бессонница и нарушения сна',
  'Сложные перемены и жизненные ситуации',
  'Эмоциональное и физическое насилие',
  'Навязчивые мысли и состояния',
  'Отношения с партнером',
  'Отношения с детьми',
  'Расставание',
  'Смысл жизни, жизненные цели, самоопределение',
  'Трудоголизм и work-life balance',
  'Работа с эмоциями',
  'Страхи',
  'Недовольство своим телом и внешностью',
  'Тяжелое физическое заболевание',
  'Трудный прошлый опыт',
  'Одиночество и отсутствие отношений',
  'Отношения с родителями',
  'Отношения с друзьями, коллегами и другими людьми',
  'Потеря близкого',
  'Прокрастинация и личная эффективность',
  'Тревога'],
 'С кем работает': ['Взрослые'],
 'Опыт работы (в годах)': 7,
 'Возраст (в годах)': 45,
 'Семейное по

In [ ]:
import pandas as pd

data_list = []
for url in tqdm(read_links('links.txt'), desc="Парсинг страниц психологов", unit="страница"):
    try:
        data = parse_psychologist_page(url)
        data_list.append(data)
    except Exception as e:
        print(f"Ошибка при обработке ссылки {url}: {e}")
        error.append(url)
df = pd.DataFrame(data_list)

df.to_csv('psychologists_data.csv', index=False, encoding='utf-8')

df.head(20)

Парсинг страниц психологов:  27%|██▋       | 236/877 [44:11<1:50:26, 10.34s/страница]

Ошибка при обработке ссылки https://alter.ru/psychologists/irkutsk/ekaterina-spiridonova: Message: 



Парсинг страниц психологов:  35%|███▍      | 303/877 [56:22<1:33:06,  9.73s/страница]

Ошибка при обработке ссылки https://alter.ru/psychologists/spb/anna-eliseeva: Message: 



Парсинг страниц психологов:  35%|███▍      | 304/877 [56:33<1:36:54, 10.15s/страница]

...

Ошибка при обработке ссылки https://alter.ru/psychologists/voronezh/lila-ponomareva: Message: 



Парсинг страниц психологов:  71%|███████▏  | 627/877 [1:58:21<49:21, 11.85s/страница]

Ошибка при обработке ссылки https://alter.ru/psychologists/msk/olga-goriushkina: Message: 



Парсинг страниц психологов: 100%|██████████| 877/877 [2:46:06<00:00, 11.36s/страница]


,Имя фамилия,Цена за сессию (руб),Город,Онлайн,Лично,Подходы,Специализация,С кем работает,Опыт работы (в годах),Возраст (в годах),...,Пол,Наличие детей,Работает ли с верующими,Фем-френдли,Квир-френдли,Работает ли с онко-пациентами,Образование,Наличие видеоприветствия,Наличие аудиоприветствия,О себе
0,Земфира Максимова,3500,Калининград,1,1,[Гештальт-терапия],"[Самооценка и уверенность в себе, Сложные пере...",[Взрослые],4,45.0,...,0,0.0,0,0,0,0,[АНО «Национальный Открытый Институт г. Санкт-...,1,0,"Здравствуйте меня зовут Земфира, я психолог, г..."
1,Алина Иванова,5000,Москва,1,0,"[КПТ, ACT]","[Самооценка и уверенность в себе, Страхи, Смыс...",[Взрослые],7,34.0,...,0,1.0,0,0,0,0,[Московский институт психоанализа. «Психология...,0,1,"Верю в силы, возможности и способности человек..."
2,Светлана Матякубова,5000,Санкт-Петербург,1,1,[Психоанализ],"[Самооценка и уверенность в себе, Трудный прош...",[Взрослые],17,57.0,...,0,1.0,0,0,1,0,[НОУ «Восточно-Европейский Институт Психоанали...,0,0,"Я психолог, психоаналитической ориентации, спе..."
3,Екатерина Новицкая,5000,Москва,1,0,"[КПТ, РЭПТ]","[Самооценка и уверенность в себе, Страхи, Отно...",[Взрослые],5,29.0,...,0,0.0,0,0,0,0,[Московский педагогический государственный уни...,1,0,"Здравствуйте! Меня зовут Екатерина, я психолог..."
4,Елена Щеблыкина,4000,Томск,1,0,[КПТ],"[Самооценка и уверенность в себе, Эмоционально...",[Взрослые],5,28.0,...,0,0.0,0,0,0,0,[Сибирский государственный медицинский универс...,0,0,В своей практике опираюсь на проверенные метод...
5,Виталий Волков,5000,Москва,1,1,"[Транзактный анализ, Понимающая]","[Самооценка и уверенность в себе, Трудный прош...","[Взрослые, Люди в возрасте]",10,31.0,...,1,0.0,0,0,1,0,[Московский городской психолого-педагогический...,0,0,"Я искренне верю в клиента, вижу и поддерживаю ..."
6,Ольга Дятлова,9000,Москва,1,0,"[КПТ, MBCT, Схематерапия]","[Самооценка и уверенность в себе, Эмоционально...","[Взрослые, Семьи]",8,35.0,...,0,0.0,0,0,1,1,[РНИМУ им. Н.И. Пирогова. «Клинический психоло...,0,0,Здравствуйте и приятно познакомиться! Большую ...
7,Кристина Сапожникова,6000,Санкт-Петербург,1,0,"[КПТ, Схематерапия]","[Самооценка и уверенность в себе, Страхи, Псих...",[Взрослые],6,29.0,...,0,0.0,0,0,1,0,[Санкт-Петербургский государственный университ...,0,0,"Я часто сталкиваюсь с тем, что люди относятся ..."
8,Лилия Сидоренко,4300,Краснодар,1,0,[КПТ],"[Самооценка и уверенность в себе, Эмоционально...","[Взрослые, Люди в возрасте]",3,32.0,...,0,0.0,0,0,0,0,[Кубанский Государственный Университет. «Психо...,1,0,Здравствуйте! Меня зовут — Лилия. Поиск психол...
9,Маргарита Спасская,9000,Тель-Авив,1,0,[Схематерапия],"[Самооценка и уверенность в себе, Отношения с ...","[Взрослые, Люди в возрасте]",17,NaN,...,0,NaN,0,0,1,0,[Восточно-Европейский Институт Психоанализа. К...,1,0,"Пауза. То, что часто необходимо нам, чтобы зад..."


Допарсим те странички что не загрузились в основном парсинге

In [ ]:
error_repeat = []
error_data = []
for url in tqdm(error, desc="Парсинг страниц психологов", unit="страница"):
    try:
        data = parse_psychologist_page(url)
        error_data.append(data)
    except Exception as e:
        print(f"Ошибка при обработке ссылки {url}: {e}")
        error_repeat.append(url)
df_rep = pd.DataFrame(error_data)

df_rep

Парсинг страниц психологов:  89%|████████▉ | 17/19 [03:27<00:24, 12.14s/страница]

Ошибка при обработке ссылки https://alter.ru/psychologists/msk/anna-cherkesova: Message: 



Парсинг страниц психологов:  95%|█████████▍| 18/19 [03:37<00:11, 11.69s/страница]

Ошибка при обработке ссылки https://alter.ru/psychologists/voronezh/lila-ponomareva: Message: 



Парсинг страниц психологов: 100%|██████████| 19/19 [03:48<00:00, 12.04s/страница]

Ошибка при обработке ссылки https://alter.ru/psychologists/msk/olga-goriushkina: Message: 



,Имя фамилия,Цена за сессию (руб),Город,Онлайн,Лично,Подходы,Специализация,С кем работает,Опыт работы (в годах),Возраст (в годах),...,Пол,Наличие детей,Работает ли с верующими,Фем-френдли,Квир-френдли,Работает ли с онко-пациентами,Образование,Наличие видеоприветствия,Наличие аудиоприветствия,О себе
0,Екатерина Спиридонова,2500,Иркутск,1,0,[Гештальт-терапия],"[Самооценка и уверенность в себе, Эмоционально...",[Взрослые],5,37,...,0,1.0,0,0,0,0,[Иркутский Государственный Университет програм...,1,0,"Меня зовут Спиридонова Екатерина, я психолог, ..."
1,Анна Елисеева,10000,Санкт-Петербург,1,0,"[Психоанализ, ТФП]","[Самооценка и уверенность в себе, Тяжелое физи...","[Взрослые, Люди в возрасте]",19,46,...,0,NaN,0,0,1,0,[США. Смит Колледж Клинической Социальной рабо...,1,0,Моя профессия с 2007 года – помогать людям пре...
2,Дарья Останкова,3950,Москва,1,0,"[Гештальт-терапия, Юнгианский анализ]","[Зависимости, Отношения с партнером, Отношения...",[Взрослые],9,40,...,0,0.0,0,0,0,0,[Московский Государственный Университет им. М....,0,0,Мой основной принцип — «Человеку нужен человек...
3,Анастасия Иванова,4000,Москва,1,0,"[ССТ, ЭФТ]","[Самооценка и уверенность в себе, Недовольство...","[Взрослые, Семьи]",2,30,...,0,1.0,0,0,0,0,[МГУ имени М. В. Ломоносова. «Психология перег...,0,1,Помогаю обрести устойчивость в непростых жизне...
4,Любовь Колташова,4500,Санкт-Петербург,1,1,"[Схематерапия, EMDR, КЦП]","[Самооценка и уверенность в себе, Недовольство...",[Взрослые],8,32,...,0,0.0,0,0,1,0,[Уральский федеральный университет им Б.Н. Ель...,0,0,"Психотерапия — это возможность жить ту жизнь, ..."
5,Юлия Светличкина,6500,Самара,1,0,[Психоанализ],"[Самооценка и уверенность в себе, Отношения с ...",[Взрослые],14,39,...,0,1.0,1,0,0,0,[Самарский Государственный Университет. Факуль...,0,0,"Здравствуйте! Я, Юлия, психолог, влюбленный в ..."
6,Мария Каллер Салас,4500,Москва,1,0,"[КПТ, Психоанализ, Транзактный анализ]","[Самооценка и уверенность в себе, Эмоционально...",[Взрослые],4,43,...,0,1.0,0,0,0,0,"[МГУ Ломоносова. «Социология» - Социолог, преп...",0,0,"С тех пор, как впервые психотерапия мне помогл..."
7,Мария Минакова,4500,Москва,1,1,"[Психодрама, Понимающая]","[Самооценка и уверенность в себе, Эмоционально...",[Взрослые],15,59,...,0,1.0,1,0,0,0,[МГППУ. «Консультативная и клиническая психоло...,1,0,Своей профессиональной миссией вижу помощь чел...
8,Эльвира Моисеева,6000,Сочи,1,1,"[КПТ, ССТ]","[Самооценка и уверенность в себе, Психические ...","[Взрослые, Семьи]",17,39,...,0,1.0,0,0,0,0,[Челябинский Государственный Педагогический ун...,0,0,Мои клиенты и коллеги характеризуют меня как б...
9,Елена Курдюкова,5000,Москва,1,1,[Психоанализ],"[Работа с эмоциями, Отношения с партнером, Отн...",[Взрослые],20,40,...,0,1.0,0,0,0,0,[Российский государственный медицинский универ...,0,0,"Я клинический психолог, получила образование в..."


In [ ]:
error_repeat2 = []

for url in tqdm(error_repeat, desc="Парсинг страниц психологов", unit="страница"):
    try:
        data = parse_psychologist_page(url)
        error_data.append(data)
    except Exception as e:
        print(f"Ошибка при обработке ссылки {url}: {e}")
        error_repeat2.append(url)

df_rep = pd.DataFrame(error_data)
df_rep

Парсинг страниц психологов: 100%|██████████| 3/3 [00:29<00:00,  9.94s/страница]


,Имя фамилия,Цена за сессию (руб),Город,Онлайн,Лично,Подходы,Специализация,С кем работает,Опыт работы (в годах),Возраст (в годах),...,Пол,Наличие детей,Работает ли с верующими,Фем-френдли,Квир-френдли,Работает ли с онко-пациентами,Образование,Наличие видеоприветствия,Наличие аудиоприветствия,О себе
0,Екатерина Спиридонова,2500,Иркутск,1,0,[Гештальт-терапия],"[Самооценка и уверенность в себе, Эмоционально...",[Взрослые],5,37,...,0,1.0,0,0,0,0,[Иркутский Государственный Университет програм...,1,0,"Меня зовут Спиридонова Екатерина, я психолог, ..."
1,Анна Елисеева,10000,Санкт-Петербург,1,0,"[Психоанализ, ТФП]","[Самооценка и уверенность в себе, Тяжелое физи...","[Взрослые, Люди в возрасте]",19,46,...,0,NaN,0,0,1,0,[США. Смит Колледж Клинической Социальной рабо...,1,0,Моя профессия с 2007 года – помогать людям пре...
2,Дарья Останкова,3950,Москва,1,0,"[Гештальт-терапия, Юнгианский анализ]","[Зависимости, Отношения с партнером, Отношения...",[Взрослые],9,40,...,0,0.0,0,0,0,0,[Московский Государственный Университет им. М....,0,0,Мой основной принцип — «Человеку нужен человек...
3,Анастасия Иванова,4000,Москва,1,0,"[ССТ, ЭФТ]","[Самооценка и уверенность в себе, Недовольство...","[Взрослые, Семьи]",2,30,...,0,1.0,0,0,0,0,[МГУ имени М. В. Ломоносова. «Психология перег...,0,1,Помогаю обрести устойчивость в непростых жизне...
4,Любовь Колташова,4500,Санкт-Петербург,1,1,"[Схематерапия, EMDR, КЦП]","[Самооценка и уверенность в себе, Недовольство...",[Взрослые],8,32,...,0,0.0,0,0,1,0,[Уральский федеральный университет им Б.Н. Ель...,0,0,"Психотерапия — это возможность жить ту жизнь, ..."
5,Юлия Светличкина,6500,Самара,1,0,[Психоанализ],"[Самооценка и уверенность в себе, Отношения с ...",[Взрослые],14,39,...,0,1.0,1,0,0,0,[Самарский Государственный Университет. Факуль...,0,0,"Здравствуйте! Я, Юлия, психолог, влюбленный в ..."
6,Мария Каллер Салас,4500,Москва,1,0,"[КПТ, Психоанализ, Транзактный анализ]","[Самооценка и уверенность в себе, Эмоционально...",[Взрослые],4,43,...,0,1.0,0,0,0,0,"[МГУ Ломоносова. «Социология» - Социолог, преп...",0,0,"С тех пор, как впервые психотерапия мне помогл..."
7,Мария Минакова,4500,Москва,1,1,"[Психодрама, Понимающая]","[Самооценка и уверенность в себе, Эмоционально...",[Взрослые],15,59,...,0,1.0,1,0,0,0,[МГППУ. «Консультативная и клиническая психоло...,1,0,Своей профессиональной миссией вижу помощь чел...
8,Эльвира Моисеева,6000,Сочи,1,1,"[КПТ, ССТ]","[Самооценка и уверенность в себе, Психические ...","[Взрослые, Семьи]",17,39,...,0,1.0,0,0,0,0,[Челябинский Государственный Педагогический ун...,0,0,Мои клиенты и коллеги характеризуют меня как б...
9,Елена Курдюкова,5000,Москва,1,1,[Психоанализ],"[Работа с эмоциями, Отношения с партнером, Отн...",[Взрослые],20,40,...,0,1.0,0,0,0,0,[Российский государственный медицинский универ...,0,0,"Я клинический психолог, получила образование в..."


Ура!! Мы наокнец-то справились со **всеми** ссылками

Записываем в итоговый файл все вместе

In [ ]:
df_final = pd.DataFrame(data_list + error_data)

df_final.to_csv('psychologists_data.csv', index=False, encoding='utf-8')

In [ ]:
df_final.shape

(877, 21)

In [ ]:
df_final.to_excel('psychologists.xlsx', index=False) #просто чтобы легче было за данными подсматривать

# Проверили, что все данные на месте - слава богу!!

Парсер готов, радуемся, гордимся!!😋😋😋

<a href="https://ibb.co/WpvHwg5M"><img src="https://i.ibb.co/tPpHt8Kv/IMG-5593.jpg" alt="IMG-5593" border="0"></a>